In [1]:
import pandas as pd
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', None) 
import numpy as np
import re
from collections import defaultdict

In [2]:
old_depth = pd.read_csv('Raw/depth_14to24.csv')
old_depth.head()

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
0,2014,ATL,2.0,REG,2,Umenyiora,Osi,Osi,Defense,00-0021977,50,DE,UME444955,DE,Osi Umenyiora
1,2014,ATL,2.0,REG,3,Maponga,Stansly,Stansly,Defense,00-0030425,59,DE,MAP697687,DE,Stansly Maponga
2,2014,ATL,2.0,REG,1,Jackson,Tyson,Tyson,Defense,00-0026980,94,DE,JAC610518,DT,Tyson Jackson
3,2014,ATL,2.0,REG,2,Hageman,Ra'Shede,Ra'Shede,Defense,00-0031281,90,DT,HAG152196,DT,Ra'Shede Hageman
4,2014,ATL,2.0,REG,3,Matthews,Cliff,Cliff,Defense,00-0028168,98,DE,MAT000000,DT,Cliff Matthews


In [3]:
old_depth = old_depth.loc[((old_depth['position'].isin(['QB', 'RB', 'FB', 'TE', 'WR'])) & (old_depth['formation'] == 'Offense')) | (old_depth['position'] == 'K')]

In [4]:
old_depth.shape

(111899, 15)

In [5]:
old_depth = old_depth.loc[
    (old_depth['week'].isin([1, 17, 18]) & 
     ~((old_depth['club_code'].isin(['TB', 'MIA'])) & (old_depth['season'] == 2017))) |
    (old_depth['club_code'].isin(['TB', 'MIA']) & 
     (old_depth['season'] == 2017) & 
     old_depth['week'].isin([2, 17, 18]))
]


In [6]:
old_depth[(old_depth['club_code'] == 'MIA') & (old_depth['position'] == 'RB')].head(50)

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
14732,2014,MIA,1.0,REG,1,Miller,Lamar,Lamar,Offense,00-0029615,36,RB,MIL373118,RB,Lamar Miller
14739,2014,MIA,1.0,REG,3,Williams,Damien,Damien,Offense,00-0030874,29,RB,WIL206455,RB,Damien Williams
14740,2014,MIA,1.0,REG,3,Darkwa,Orleans,Orleans,Offense,00-0030860,26,RB,DAR352254,\n,Orleans Darkwa
14741,2014,MIA,1.0,REG,2,Moreno,Knowshon,Knowshon,Offense,00-0026988,28,RB,MOR120768,RB,Knowshon Moreno
15588,2014,MIA,17.0,REG,1,Miller,Lamar,Lamar,Offense,00-0029615,36,RB,MIL373118,RB,Lamar Miller
15595,2014,MIA,17.0,REG,2,Thomas,Daniel,Daniel,Offense,00-0028000,30,RB,THO092066,RB,Daniel Thomas
15598,2014,MIA,17.0,REG,3,Williams,Damien,Damien,Offense,00-0030874,29,RB,WIL206455,RB,Damien Williams
15599,2014,MIA,17.0,REG,3,James,LaMichael,LaMichael,Offense,00-0029260,27,RB,JAM321847,\n,LaMichael James
48689,2015,MIA,1.0,REG,1,Miller,Lamar,Lamar,Offense,00-0029615,36,RB,MIL373118,RB,Lamar Miller
48695,2015,MIA,1.0,REG,2,Williams,Damien,Damien,Offense,00-0030874,29,RB,WIL206455,RB,Damien Williams


In [7]:
old_depth['club_code'] = old_depth['club_code'].replace({'OAK': 'LV', 'SD': 'LAC', 'STL' : 'LA'})

In [8]:
old_depth.loc[((old_depth['club_code'].isin(['MIA', 'TB'])) & (old_depth['week'] == 2) & (old_depth['season'] == 2017)), 'week'] = 1

In [9]:
old_depth.loc[((old_depth['club_code'].isin(['MIA', 'TB'])) & (old_depth['week'] == 1) & (old_depth['season'] == 2017))]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
122830,2017,MIA,1.0,REG,3,Perry,Senorise,Senorise,Offense,00-0031209,20,RB,PER713312,\n,Senorise Perry
122833,2017,MIA,1.0,REG,1,Thomas,Julius,Julius,Offense,00-0028067,89,TE,THO265401,TE,Julius Thomas
122839,2017,MIA,1.0,REG,3,Gray,MarQueis,MarQueis,Offense,00-0029878,48,TE,GRA739815,TE,MarQueis Gray
122846,2017,MIA,1.0,REG,1,Landry,Jarvis,Jarvis,Offense,00-0031382,5,WR,LAN163029,WR,Jarvis Landry
122849,2017,MIA,1.0,REG,2,Carroo,Leonte,Leonte,Offense,00-0033060,88,WR,CAR559411,WR,Leonte Carroo
122854,2017,MIA,1.0,REG,2,Fasano,Anthony,Anthony,Offense,00-0024268,80,TE,FAS072076,TE,Anthony Fasano
122855,2017,MIA,1.0,REG,1,Cutler,Jay,Jay,Offense,00-0024226,6,QB,CUT288111,QB,Jay Cutler
122857,2017,MIA,1.0,REG,1,Stills,Kenneth,Kenny,Offense,00-0030085,12,WR,STI417540,WR,Kenny Stills
122858,2017,MIA,1.0,REG,1,Ajayi,Jay,Jay,Offense,00-0031590,28,RB,AJA771256,RB,Jay Ajayi
122863,2017,MIA,1.0,REG,3,Drake,Kenyan,Kenyan,Offense,00-0033118,32,RB,DRA280410,RB,Kenyan Drake


In [10]:
old_depth['club_code'].value_counts().count()

np.int64(32)

In [11]:
old_depth.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17919 entries, 632 to 401574
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   season          17919 non-null  int64  
 1   club_code       17919 non-null  object 
 2   week            17919 non-null  float64
 3   game_type       17919 non-null  object 
 4   depth_team      17919 non-null  int64  
 5   last_name       17919 non-null  object 
 6   first_name      17919 non-null  object 
 7   football_name   17919 non-null  object 
 8   formation       17919 non-null  object 
 9   gsis_id         17919 non-null  object 
 10  jersey_number   17919 non-null  int64  
 11  position        17919 non-null  object 
 12  elias_id        17914 non-null  object 
 13  depth_position  17919 non-null  object 
 14  full_name       17919 non-null  object 
dtypes: float64(1), int64(3), object(11)
memory usage: 2.2+ MB


In [12]:
# old_depth.loc[((old_depth['position'].isin(['QB', 'RB', 'FB', 'WR', 'TE'])) & (old_depth['formation'] == 'Offense')) | (old_depth['position'] == 'K')]

In [13]:
old_depth.duplicated().sum()

np.int64(69)

In [14]:
old_depth = old_depth.drop_duplicates()

In [15]:
old_depth[old_depth['full_name'] == 'Adam Prentice']

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
272163,2021,NO,1.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
273101,2021,NO,17.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
273155,2021,NO,18.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
309768,2022,NO,18.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
309834,2022,NO,1.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
347957,2023,NO,1.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
348194,2023,NO,17.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
348227,2023,NO,18.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
384461,2024,NO,1.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
385421,2024,NO,17.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice


In [16]:
old_depth = old_depth.loc[((old_depth['season'] >= 2021) & (old_depth['week'].isin([1,18]))) | ((old_depth['season'] <= 2020) & (old_depth['week'].isin([1, 17])))]

In [17]:
old_depth[old_depth['full_name'] == 'Adam Prentice']

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
272163,2021,NO,1.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
273155,2021,NO,18.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
309768,2022,NO,18.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
309834,2022,NO,1.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
347957,2023,NO,1.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
348227,2023,NO,18.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
384461,2024,NO,1.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice
385516,2024,NO,18.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice


In [18]:
old_depth.head()

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
632,2014,ATL,1.0,REG,1,White,Sharod,Roddy,Offense,00-0023462,84,WR,WHI472686,WR,Roddy White
636,2014,ATL,1.0,REG,1,Jones,Quintorris,Julio,Offense,00-0027944,80,WR,JON640315,WR,Julio Jones
637,2014,ATL,1.0,REG,1,Ryan,Matthew,Matt,Offense,00-0026143,2,QB,RYA238179,QB,Matt Ryan
638,2014,ATL,1.0,REG,1,Jackson,Steven,Steven,Offense,00-0022736,39,RB,JAC560875,RB,Steven Jackson
639,2014,ATL,1.0,REG,2,Douglas,Harry,Harry,Offense,00-0026224,83,WR,DOU372712,WR,Harry Douglas


In [19]:
old_depth['gsis_unq'] = old_depth['gsis_id'] + '_' + old_depth['season'].astype(str)
old_depth.head()

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq
632,2014,ATL,1.0,REG,1,White,Sharod,Roddy,Offense,00-0023462,84,WR,WHI472686,WR,Roddy White,00-0023462_2014
636,2014,ATL,1.0,REG,1,Jones,Quintorris,Julio,Offense,00-0027944,80,WR,JON640315,WR,Julio Jones,00-0027944_2014
637,2014,ATL,1.0,REG,1,Ryan,Matthew,Matt,Offense,00-0026143,2,QB,RYA238179,QB,Matt Ryan,00-0026143_2014
638,2014,ATL,1.0,REG,1,Jackson,Steven,Steven,Offense,00-0022736,39,RB,JAC560875,RB,Steven Jackson,00-0022736_2014
639,2014,ATL,1.0,REG,2,Douglas,Harry,Harry,Offense,00-0026224,83,WR,DOU372712,WR,Harry Douglas,00-0026224_2014


In [20]:
# wk1 = old_depth.loc[old_depth['week'] == 1, ['gsis_unq', 'depth_team']]
# wk1 = wk1.rename(columns={'depth_team': 'wk1_depth'})

In [21]:
# eos = old_depth.loc[old_depth['week'].isin([17,18])].sort_values(by='week')
# eos = eos.groupby('gsis_unq').last()[['depth_team']].reset_index()
# eos = eos.rename(columns={'depth_team': 'eos_depth'})
# eos.head()

In [22]:
# eos[eos['gsis_unq'] == '00-0022736_2014']

In [23]:
# old_depth[old_depth['full_name'] == 'Adam Prentice']

In [24]:
# old_depth = old_depth.merge(wk1, how='left', on='gsis_unq')
# old_depth = old_depth.merge(eos, how='left', on='gsis_unq')
# old_depth.head()

In [25]:
# old_depth[old_depth['full_name'] == 'Adam Prentice']

In [26]:
# old_depth[old_depth['gsis_unq'] == '00-0022736_2014']

In [27]:
# old_depth.info()

In [28]:
# old_depth[old_depth['wk1_depth'].isna()]

In [29]:
# old_depth[old_depth['eos_depth'].isna()]

In [30]:
# old_depth[['wk1_depth', 'eos_depth']] = old_depth[['wk1_depth', 'eos_depth']].fillna(99)
# old_depth.info()

In [31]:
old_depth.head(20)

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq
632,2014,ATL,1.0,REG,1,White,Sharod,Roddy,Offense,00-0023462,84,WR,WHI472686,WR,Roddy White,00-0023462_2014
636,2014,ATL,1.0,REG,1,Jones,Quintorris,Julio,Offense,00-0027944,80,WR,JON640315,WR,Julio Jones,00-0027944_2014
637,2014,ATL,1.0,REG,1,Ryan,Matthew,Matt,Offense,00-0026143,2,QB,RYA238179,QB,Matt Ryan,00-0026143_2014
638,2014,ATL,1.0,REG,1,Jackson,Steven,Steven,Offense,00-0022736,39,RB,JAC560875,RB,Steven Jackson,00-0022736_2014
639,2014,ATL,1.0,REG,2,Douglas,Harry,Harry,Offense,00-0026224,83,WR,DOU372712,WR,Harry Douglas,00-0026224_2014
643,2014,ATL,1.0,REG,2,Hester,Devin,Devin,Offense,00-0024272,17,WR,HES267217,WR,Devin Hester,00-0024272_2014
644,2014,ATL,1.0,REG,2,Yates,Taylor,T.J.,Offense,00-0028090,2,QB,YAT420091,QB,T.J. Yates,00-0028090_2014
645,2014,ATL,1.0,REG,2,Rodgers,Jacquizz,Jacquizz,Offense,00-0028083,37,RB,ROD369979,RB,Jacquizz Rodgers,00-0028083_2014
646,2014,ATL,1.0,REG,3,Weems,Eric,Eric,Offense,00-0024535,14,WR,WEE551877,WR,Eric Weems,00-0024535_2014
647,2014,ATL,1.0,REG,3,Roby,Courtney,Courtney,Offense,00-0023503,15,WR,ROB808006,WR,Courtney Roby,00-0023503_2014


In [32]:
old_depth[(old_depth['position'] == 'FB') & (old_depth['season'] == 2024)]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq
365528,2024,BUF,1.0,REG,1,Gilliam,Reggie,Reggie,Offense,00-0036187,41,FB,GIL562234,FB,Reggie Gilliam,00-0036187_2024
366576,2024,BUF,18.0,REG,1,Gilliam,Reggie,Reggie,Offense,00-0036187,41,FB,GIL562234,FB,Reggie Gilliam,00-0036187_2024
366909,2024,CHI,1.0,REG,1,Blasingame,Khari,Khari,Offense,00-0035099,48,FB,BLA772342,FB,Khari Blasingame,00-0035099_2024
369451,2024,BAL,18.0,REG,1,Ricard,Patrick,Patrick,Offense,00-0033376,42,FB,RIC012529,FB,Patrick Ricard,00-0033376_2024
369596,2024,BAL,1.0,REG,1,Ricard,Patrick,Patrick,Offense,00-0033376,42,FB,RIC012529,FB,Patrick Ricard,00-0033376_2024
370391,2024,DAL,1.0,REG,3,Luepke,Hunter,Hunter,Offense,00-0038738,40,FB,LUE582275,RB,Hunter Luepke,00-0038738_2024
371345,2024,DAL,18.0,REG,1,Luepke,Hunter,Hunter,Offense,00-0038738,40,FB,LUE582275,FB,Hunter Luepke,00-0038738_2024
371442,2024,DEN,1.0,REG,1,Burton,Michael,Michael,Offense,00-0031595,20,FB,BUR792195,FB,Michael Burton,00-0031595_2024
372458,2024,DEN,18.0,REG,1,Burton,Michael,Michael,Offense,00-0031595,20,FB,BUR792195,FB,Michael Burton,00-0031595_2024
381463,2024,MIA,18.0,REG,1,Ingold,Alec,Alec,Offense,00-0035125,30,FB,ING382661,F,Alec Ingold,00-0035125_2024


In [33]:
coach_stats = pd.read_csv('stats_no_depth.csv')
coach_stats.head()

,gsis_id,cleaned_name_yahoo,player_key,primary_position,season,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,team,def_special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_safeties,def_misc_yards,def_fumble_recovery_opp,def_fumble_recovery_yards_opp,def_fumble_recovery_tds,def_punt_returns,def_punt_return_yards,def_kickoff_returns,def_kickoff_return_yards,HC,OC,DC
0,00-0004091,phil dawson,331.p.4269,K,2014,00-0004091_2014,4.0,125.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,31.0,5.0,1.0,55.0,0.806452,0.0,6.0,8.0,5.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948.0,250.0,45.0,33.0,33.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,san francisco 49ers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jim harbaugh,greg roman,vic fangio
1,00-0004091,phil dawson,348.p.4269,K,2015,00-0004091_2015,9.0,102.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,27.0,1.0,2.0,54.0,0.888889,0.0,9.0,6.0,6.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30;25;47;33;43;22;53;31;26;42;35;54;26;44;27;2...,45,28;41,848.0,45.0,69.0,20.0,21.0,0.0,1.0,0.952381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,san francisco 49ers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jim tomsula,geep chryst,eric mangini
2,00-0004091,phil dawson,359.p.4269,K,2016,00-0004091_2016,9.0,95.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,21.0,3.0,0.0,53.0,0.857143,0.0,4.0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,29;39;22;36;33;48;47;28;49;48;46;49;53;33;36;3...,53;44;48,0,702.0,145.0,0.0,33.0,34.0,1.0,0.0,0.970588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,san franci

In [34]:
coach_stats['gs_unq'].nunique()

7720

In [35]:
old_depth['gsis_unq'].nunique()

6732

In [36]:
coach_stats.loc[~coach_stats['gs_unq'].isin(old_depth['gsis_unq']) & (coach_stats['primary_position'] != 'DEF') & (coach_stats['season'] != 2025)].head()

,gsis_id,cleaned_name_yahoo,player_key,primary_position,season,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,team,def_special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_safeties,def_misc_yards,def_fumble_recovery_opp,def_fumble_recovery_yards_opp,def_fumble_recovery_tds,def_punt_returns,def_punt_return_yards,def_kickoff_returns,def_kickoff_return_yards,HC,OC,DC
55,00-0020972,billy cundiff,348.p.6154,K,2015,00-0020972_2015,8.0,0.0,BUF,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,buffalo bills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,rex ryan,greg roman,dennis thurman
71,00-0021570,kassim osgood,331.p.6664,WR,2014,00-0021570_2014,8.0,0.0,SF,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,-0.363759,0.0,0.000000,0.002066,0.001056,0.003838,0.0,2.0,25.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,san francisco 49ers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jim harbaugh,greg roman,vic fangio
77,00-0021940,josh brown,359.p.6558,K,2016,00-0021940_2016,8.0,46.0,NYG,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,11.0,12.0,1.0,0.0,48.0,0.916667,1.0,3.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,48;19;23;29;30;40;47;41;30;21;31,53,0,359.0,53.0,0.0,9.0,9.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,new york giants,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Non Overlap looks to be 2025 players as well as players who only played weeks mid season and werent on the depth chart at beginnign or end. Will impute these players depth positons with 99 once merge is done

In [37]:
old_depth.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11644 entries, 632 to 401574
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   season          11644 non-null  int64  
 1   club_code       11644 non-null  object 
 2   week            11644 non-null  float64
 3   game_type       11644 non-null  object 
 4   depth_team      11644 non-null  int64  
 5   last_name       11644 non-null  object 
 6   first_name      11644 non-null  object 
 7   football_name   11644 non-null  object 
 8   formation       11644 non-null  object 
 9   gsis_id         11644 non-null  object 
 10  jersey_number   11644 non-null  int64  
 11  position        11644 non-null  object 
 12  elias_id        11641 non-null  object 
 13  depth_position  11644 non-null  object 
 14  full_name       11644 non-null  object 
 15  gsis_unq        11644 non-null  object 
dtypes: float64(1), int64(3), object(12)
memory usage: 1.5+ MB


In [38]:
old_depth[old_depth['full_name'] == 'Phil Dawson']

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq
24601,2014,SF,1.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2014
25467,2014,SF,17.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2014
60260,2015,SF,1.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2015
61167,2015,SF,17.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2015
97337,2016,SF,1.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2016
98306,2016,SF,17.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2016
130605,2017,ARI,1.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2017
131554,2017,ARI,17.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2017
167806,2018,ARI,1.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2018


In [39]:
old_depth.loc[~old_depth['gsis_unq'].isin(coach_stats['gs_unq'])]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq
648,2014,ATL,1.0,REG,3,Renfree,Sean,Sean,Offense,00-0030569,13,QB,REN135405,QB,Sean Renfree,00-0030569_2014
966,2014,BUF,1.0,REG,1,Summers,Frank,Frank,Offense,00-0026926,38,FB,SUM438237,FB,Frank Summers,00-0026926_2014
1943,2014,BUF,17.0,REG,3,Thompson,Deonte,Deonte,Offense,00-0029388,10,WR,THO528909,\n,Deonte Thompson,00-0029388_2014
2648,2014,CHI,1.0,REG,1,Fiammetta,Placido,Tony,Offense,00-0027093,43,FB,FIA415290,FB,Tony Fiammetta,00-0027093_2014
2653,2014,CHI,1.0,REG,3,Spurlock,Micheal,Micheal,Offense,00-0023968,18,WR,SPU401416,WR,Micheal Spurlock,00-0023968_2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396247,2024,WAS,1.0,REG,3,Driskel,Jeffrey,Jeff,Offense,00-0032436,16,QB,DRI392790,QB,Jeff Driskel,00-0032436_2024
397267,2024,WAS,18.0,REG,3,Driskel,Jeffrey,Jeff,Offense,00-0032436,16,QB,DRI392790,QB,Jeff Driskel,00-0032436_2024
397604,2024,CAR,1.0,REG,3,Swinson,Messiah,Messiah,Offense,00-0039275,48,TE,SWI549130,TE,Messiah Swinson,00-0039275_2024
401548,2024,HOU,18.0,REG,2,Wayne,Jared,Jared,Offense,00-0038728,14,WR,WAY428537,WR,Jared Wayne,00-0038728_2024


In [40]:
old_depth[(old_depth['club_code'] == 'ATL') & (old_depth['season'] == 2014)]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq
632,2014,ATL,1.0,REG,1,White,Sharod,Roddy,Offense,00-0023462,84,WR,WHI472686,WR,Roddy White,00-0023462_2014
636,2014,ATL,1.0,REG,1,Jones,Quintorris,Julio,Offense,00-0027944,80,WR,JON640315,WR,Julio Jones,00-0027944_2014
637,2014,ATL,1.0,REG,1,Ryan,Matthew,Matt,Offense,00-0026143,2,QB,RYA238179,QB,Matt Ryan,00-0026143_2014
638,2014,ATL,1.0,REG,1,Jackson,Steven,Steven,Offense,00-0022736,39,RB,JAC560875,RB,Steven Jackson,00-0022736_2014
639,2014,ATL,1.0,REG,2,Douglas,Harry,Harry,Offense,00-0026224,83,WR,DOU372712,WR,Harry Douglas,00-0026224_2014
643,2014,ATL,1.0,REG,2,Hester,Devin,Devin,Offense,00-0024272,17,WR,HES267217,WR,Devin Hester,00-0024272_2014
644,2014,ATL,1.0,REG,2,Yates,Taylor,T.J.,Offense,00-0028090,2,QB,YAT420091,QB,T.J. Yates,00-0028090_2014
645,2014,ATL,1.0,REG,2,Rodgers,Jacquizz,Jacquizz,Offense,00-0028083,37,RB,ROD369979,RB,Jacquizz Rodgers,00-0028083_2014
646,2014,ATL,1.0,REG,3,Weems,Eric,Eric,Offense,00-0024535,14,WR,WEE551877,WR,Eric Weems,00-0024535_2014
647,2014,ATL,1.0,REG,3,Roby,Courtney,Courtney,Offense,00-0023503,15,WR,ROB808006,WR,Courtney Roby,00-0023503_2014


In [41]:
grouped_depths = old_depth.groupby(['season', 'club_code', 'position', 'week'])['depth_team'].max()
grouped_depths.head()

season  club_code  position  week
2014    ARI        K         1.0     1
                             17.0    1
                   QB        1.0     2
                             17.0    3
                   RB        1.0     3
Name: depth_team, dtype: int64

In [42]:
grouped_depths = grouped_depths.reset_index()

In [43]:
grouped_depths = grouped_depths.loc[grouped_depths['position'] == 'RB']

In [44]:
grouped_depths.head(50)

,season,club_code,position,week,depth_team
4,2014,ARI,RB,1.0,3
5,2014,ARI,RB,17.0,3
16,2014,ATL,RB,1.0,3
17,2014,ATL,RB,17.0,3
28,2014,BAL,RB,1.0,3
29,2014,BAL,RB,17.0,3
39,2014,BUF,RB,1.0,3
40,2014,BUF,RB,17.0,3
51,2014,CAR,RB,1.0,3
52,2014,CAR,RB,17.0,2


In [45]:
grouped_depths.duplicated().sum()

np.int64(0)

In [46]:
old_depth[old_depth['full_name'] == 'Adam Prentice']

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq
272163,2021,NO,1.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2021
273155,2021,NO,18.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2021
309768,2022,NO,18.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2022
309834,2022,NO,1.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2022
347957,2023,NO,1.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2023
348227,2023,NO,18.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2023
384461,2024,NO,1.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2024
385516,2024,NO,18.0,REG,1,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2024


In [47]:
grouped_depths = grouped_depths.rename(columns={'depth_team' : 'max_rb'})

In [48]:
fb_depth = old_depth[old_depth['position'] == 'FB'].merge(grouped_depths, how='left', on=['season', 'club_code', 'week'])
fb_depth.duplicated().sum()

np.int64(0)

In [49]:
fb_depth['depth_team'] = fb_depth['max_rb'] + 1
fb_depth.head()

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position_x,elias_id,depth_position,full_name,gsis_unq,position_y,max_rb
0,2014,ATL,1.0,REG,4,DiMarco,Patrick,Patrick,Offense,00-0028581,42,FB,DIM110572,FB,Patrick DiMarco,00-0028581_2014,RB,3
1,2014,ATL,17.0,REG,4,DiMarco,Patrick,Patrick,Offense,00-0028581,42,FB,DIM110572,FB,Patrick DiMarco,00-0028581_2014,RB,3
2,2014,BUF,1.0,REG,4,Summers,Frank,Frank,Offense,00-0026926,38,FB,SUM438237,FB,Frank Summers,00-0026926_2014,RB,3
3,2014,CHI,1.0,REG,4,Fiammetta,Placido,Tony,Offense,00-0027093,43,FB,FIA415290,FB,Tony Fiammetta,00-0027093_2014,RB,3
4,2014,CHI,17.0,REG,4,Owens,Montell,Montell,Offense,00-0024103,24,FB,OWE687716,FB,Montell Owens,00-0024103_2014,RB,3


In [50]:
fb_depth[fb_depth.duplicated(keep=False)]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position_x,elias_id,depth_position,full_name,gsis_unq,position_y,max_rb


In [51]:
32*11*2

704

In [52]:
fb_map = fb_depth.set_index('gsis_unq')['depth_team']

In [53]:
fb_map

gsis_unq
00-0028581_2014    4
00-0028581_2014    4
00-0026926_2014    4
00-0027093_2014    4
00-0024103_2014    4
                  ..
00-0036727_2024    4
00-0036727_2024    4
00-0038614_2024    4
00-0029892_2024    4
00-0029892_2024    3
Name: depth_team, Length: 365, dtype: int64

In [54]:
coach_stats[coach_stats['cleaned_name_yahoo'] == 'adam prentice']

,gsis_id,cleaned_name_yahoo,player_key,primary_position,season,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,team,def_special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_safeties,def_misc_yards,def_fumble_recovery_opp,def_fumble_recovery_yards_opp,def_fumble_recovery_tds,def_punt_returns,def_punt_return_yards,def_kickoff_returns,def_kickoff_return_yards,HC,OC,DC
6221,00-0036727,adam prentice,406.p.33901,RB,2021,00-0036727_2021,11.0,5.1,NO,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0,2.0,-0.503234,0.0,3.0,3.0,16.0,0.0,0.0,0.0,1.0,15.0,1.0,-0.195035,0.0,16.000000,0.006329,0.000267,0.009681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,new orleans saints,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sean payton,pete carmichael,dennis allen
6222,00-0036727,adam prentice,414.p.33901,RB,2022,00-0036727_2022,9.0,4.8,NO,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,9.0,0.0,0.0,0.0,3.0,1.455663,0.0,3.0,3.0,9.0,0.0,1.0,0.0,-9.0,18.0,2.0,-0.327069,0.0,-1.000000,0.006085,-0.001978,0.007743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,new orleans saints,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dennis allen,pete carmichael,kris richard
6223,00-0036727,adam prentice,423.p.33901,RB,2023,00-0036727_2023,9.0,2.4,NO,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,0.0,1.0,1.0,1.0,-4.062858,0.0,2.0,3.0,12.0,0.0,0.0,0.0,11.0,8.0,0.0,-2.027484,0.0,1.090909,0.005181,0.002193,0.009307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,new orleans saints,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [55]:
grouped_depths['club_code'].value_counts()

club_code
ARI    22
ATL    22
BAL    22
BUF    22
CAR    22
CHI    22
CIN    22
CLE    22
DAL    22
DEN    22
DET    22
GB     22
HOU    22
IND    22
JAX    22
KC     22
LA     22
LAC    22
LV     22
MIA    22
MIN    22
NE     22
NO     22
NYG    22
NYJ    22
PHI    22
PIT    22
SEA    22
SF     22
TB     22
TEN    22
WAS    22
Name: count, dtype: int64

In [56]:
fb_map[fb_map.index.duplicated(keep=False)]

gsis_unq
00-0028581_2014    4
00-0028581_2014    4
00-0024103_2014    4
00-0029892_2014    4
00-0029892_2014    4
                  ..
00-0032918_2024    3
00-0036727_2024    4
00-0036727_2024    4
00-0029892_2024    4
00-0029892_2024    3
Name: depth_team, Length: 308, dtype: int64

In [57]:
old_depth[old_depth['gsis_unq'].duplicated(keep=False)].sort_values(by='gsis_unq')

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq
25467,2014,SF,17.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2014
24601,2014,SF,1.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2014
60260,2015,SF,1.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2015
61167,2015,SF,17.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2015
97337,2016,SF,1.0,REG,1,Dawson,Philip,Phil,Special Teams,00-0004091,4,K,DAW705989,K,Phil Dawson,00-0004091_2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366916,2024,CHI,1.0,REG,1,Williams,Caleb,Caleb,Offense,00-0039918,18,QB,WIL176897,QB,Caleb Williams,00-0039918_2024
366900,2024,CHI,1.0,REG,2,Odunze,Rome,Rome,Offense,00-0039919,15,WR,ODU426541,WR,Rome Odunze,00-0039919_2024
367855,2024,CHI,18.0,REG,1,Odunze,Rome,Rome,Offense,00-0039919,15,WR,ODU426541,WR,Rome Odunze,00-0039919_2024
387853,2024,NYJ,18.0,REG,2,Corley,Malachi,Malachi,Offense,00-0039920,14,WR,COR399318,WR,Malachi Corley,00-0039920_2024


In [58]:
old_depth = old_depth.merge(
    fb_depth[['gsis_unq', 'season', 'week', 'depth_team']].rename(columns={'depth_team': 'fb_depth'}),
    on=['gsis_unq', 'season', 'week'],
    how='left'
)
old_depth['depth_team'] = old_depth['fb_depth'].fillna(old_depth['depth_team'])
old_depth = old_depth.drop(columns='fb_depth')

In [59]:
old_depth[old_depth['position'] == 'FB']

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq
14,2014,ATL,1.0,REG,4.0,DiMarco,Patrick,Patrick,Offense,00-0028581,42,FB,DIM110572,FB,Patrick DiMarco,00-0028581_2014
27,2014,ATL,17.0,REG,4.0,DiMarco,Patrick,Patrick,Offense,00-0028581,42,FB,DIM110572,FB,Patrick DiMarco,00-0028581_2014
32,2014,BUF,1.0,REG,4.0,Summers,Frank,Frank,Offense,00-0026926,38,FB,SUM438237,FB,Frank Summers,00-0026926_2014
76,2014,CHI,1.0,REG,4.0,Fiammetta,Placido,Tony,Offense,00-0027093,43,FB,FIA415290,FB,Tony Fiammetta,00-0027093_2014
86,2014,CHI,17.0,REG,4.0,Owens,Montell,Montell,Offense,00-0024103,24,FB,OWE687716,FB,Montell Owens,00-0024103_2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11194,2024,NO,1.0,REG,4.0,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2024
11229,2024,NO,18.0,REG,4.0,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2024
11409,2024,LAC,18.0,REG,4.0,Matlock,Scott,Scott,Offense,00-0038614,44,FB,MAT347280,FB,Scott Matlock,00-0038614_2024
11426,2024,SF,1.0,REG,4.0,Juszczyk,Kyle,Kyle,Offense,00-0029892,44,FB,JUS812736,FB,Kyle Juszczyk,00-0029892_2024


In [60]:
wk1 = old_depth.loc[old_depth['week'] == 1, ['gsis_unq', 'depth_team']]
wk1 = wk1.rename(columns={'depth_team': 'wk1_depth'})

In [61]:
eos = old_depth.loc[old_depth['week'].isin([17,18])].sort_values(by='week')
eos = eos.groupby('gsis_unq').last()[['depth_team']].reset_index()
eos = eos.rename(columns={'depth_team': 'eos_depth'})
eos.head()

,gsis_unq,eos_depth
0,00-0004091_2014,1.0
1,00-0004091_2015,1.0
2,00-0004091_2016,1.0
3,00-0004091_2017,1.0
4,00-0007091_2014,2.0


In [62]:
eos[eos['gsis_unq'] == '00-0022736_2014']

,gsis_unq,eos_depth
80,00-0022736_2014,1.0


In [63]:
old_depth = old_depth.merge(wk1, how='left', on='gsis_unq')
old_depth = old_depth.merge(eos, how='left', on='gsis_unq')
old_depth.head()

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq,wk1_depth,eos_depth
0,2014,ATL,1.0,REG,1.0,White,Sharod,Roddy,Offense,00-0023462,84,WR,WHI472686,WR,Roddy White,00-0023462_2014,1.0,1.0
1,2014,ATL,1.0,REG,1.0,Jones,Quintorris,Julio,Offense,00-0027944,80,WR,JON640315,WR,Julio Jones,00-0027944_2014,1.0,1.0
2,2014,ATL,1.0,REG,1.0,Ryan,Matthew,Matt,Offense,00-0026143,2,QB,RYA238179,QB,Matt Ryan,00-0026143_2014,1.0,1.0
3,2014,ATL,1.0,REG,1.0,Jackson,Steven,Steven,Offense,00-0022736,39,RB,JAC560875,RB,Steven Jackson,00-0022736_2014,1.0,1.0
4,2014,ATL,1.0,REG,2.0,Douglas,Harry,Harry,Offense,00-0026224,83,WR,DOU372712,WR,Harry Douglas,00-0026224_2014,2.0,2.0


In [64]:
old_depth[old_depth['wk1_depth'].isna()]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq,wk1_depth,eos_depth
23,2014,ATL,17.0,REG,3.0,Davis,Drew,D.J.,Offense,00-0028307,19,WR,DAV286887,WR,D.J. Davis,00-0028307_2014,NaN,3.0
67,2014,BUF,17.0,REG,3.0,Thigpen,Marcus,Marcus,Offense,00-0026577,11,WR,THI675348,WR,Marcus Thigpen,00-0026577_2014,NaN,3.0
99,2014,CHI,17.0,REG,3.0,Annen,Blake,Blake,Offense,00-0030707,86,TE,ANN355964,TE,Blake Annen,00-0030707_2014,NaN,3.0
106,2014,CHI,17.0,REG,2.0,Feely,Thomas,Jay,Special Teams,00-0019770,4,K,FEE461435,FG,Jay Feely,00-0019770_2014,NaN,2.0
107,2014,CHI,17.0,REG,2.0,Feely,Thomas,Jay,Special Teams,00-0019770,4,K,FEE461435,KO,Jay Feely,00-0019770_2014,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12110,2024,CLE,18.0,REG,2.0,Woods,Michael,Mike,Offense,00-0037300,81,WR,WOO545683,WR,Mike Woods,00-0037300_2024,NaN,2.0
12117,2024,CLE,18.0,REG,3.0,Whiteheart,Blake,Blake,Offense,00-0038710,86,TE,WHI639076,TE,Blake Whiteheart,00-0038710_2024,NaN,3.0
12118,2024,CLE,18.0,REG,3.0,Zappe,Bailey,Bailey,Offense,00-0038108,2,QB,ZAP470663,QB,Bailey Zappe,00-0038108_2024,NaN,3.0
12144,2024,HOU,18.0,REG,2.0,Wayne,Jared,Jared,Offense,00-0038728,14,WR,WAY428537,WR,Jared Wayne,00-0038728_2024,NaN,2.0


In [65]:
old_depth[old_depth['eos_depth'].isna()]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq,wk1_depth,eos_depth
9,2014,ATL,1.0,REG,3.0,Roby,Courtney,Courtney,Offense,00-0023503,15,WR,ROB808006,WR,Courtney Roby,00-0023503_2014,3.0,NaN
10,2014,ATL,1.0,REG,3.0,Renfree,Sean,Sean,Offense,00-0030569,13,QB,REN135405,QB,Sean Renfree,00-0030569_2014,3.0,NaN
11,2014,ATL,1.0,REG,3.0,Smith,Antone,Antone,Offense,00-0026855,36,RB,SMI040756,RB,Antone Smith,00-0026855_2014,3.0,NaN
32,2014,BUF,1.0,REG,4.0,Summers,Frank,Frank,Offense,00-0026926,38,FB,SUM438237,FB,Frank Summers,00-0026926_2014,4.0,NaN
34,2014,BUF,1.0,REG,2.0,Williams,Mike,Mike,Offense,00-0027702,15,WR,WIL446858,WR,Mike Williams,00-0027702_2014,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12132,2024,HOU,1.0,REG,2.0,Jordan,Brevin,Brevin,Offense,00-0036556,9,TE,JOR035760,TE,Brevin Jordan,00-0036556_2024,2.0,NaN
12135,2024,HOU,1.0,REG,3.0,Brooks,British,British,Offense,00-0039603,44,RB,BRO118766,RB,British Brooks,00-0039603_2024,3.0,NaN
12136,2024,HOU,1.0,REG,3.0,Stover,Cade,Cade,Offense,00-0039359,87,TE,STO763332,TE,Cade Stover,00-0039359_2024,3.0,NaN
12137,2024,HOU,1.0,REG,1.0,Dell,Nathaniel,Tank,Offense,00-0038977,3,WR,DEL440817,WR,Tank Dell,00-0038977_2024,1.0,NaN


In [66]:
old_depth[['wk1_depth', 'eos_depth']] = old_depth[['wk1_depth', 'eos_depth']].fillna(99)
old_depth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12152 entries, 0 to 12151
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   season          12152 non-null  int64  
 1   club_code       12152 non-null  object 
 2   week            12152 non-null  float64
 3   game_type       12152 non-null  object 
 4   depth_team      12152 non-null  float64
 5   last_name       12152 non-null  object 
 6   first_name      12152 non-null  object 
 7   football_name   12152 non-null  object 
 8   formation       12152 non-null  object 
 9   gsis_id         12152 non-null  object 
 10  jersey_number   12152 non-null  int64  
 11  position        12152 non-null  object 
 12  elias_id        12149 non-null  object 
 13  depth_position  12152 non-null  object 
 14  full_name       12152 non-null  object 
 15  gsis_unq        12152 non-null  object 
 16  wk1_depth       12152 non-null  float64
 17  eos_depth       12152 non-null 

In [67]:
old_depth[old_depth['full_name'] == 'Adam Prentice']

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,gsis_unq,wk1_depth,eos_depth
8487,2021,NO,1.0,REG,3.0,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2021,3.0,4.0
8510,2021,NO,18.0,REG,4.0,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2021,3.0,4.0
9570,2022,NO,18.0,REG,4.0,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2022,4.0,4.0
9575,2022,NO,1.0,REG,4.0,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2022,4.0,4.0
10653,2023,NO,1.0,REG,4.0,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2023,4.0,3.0
10660,2023,NO,18.0,REG,3.0,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2023,4.0,3.0
11680,2024,NO,1.0,REG,4.0,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2024,4.0,4.0
11719,2024,NO,18.0,REG,4.0,Prentice,Adam,Adam,Offense,00-0036727,46,FB,PRE226147,FB,Adam Prentice,00-0036727_2024,4.0,4.0


In [68]:
old_depth = old_depth[['gsis_unq', 'wk1_depth', 'eos_depth']]
old_depth.head()

,gsis_unq,wk1_depth,eos_depth
0,00-0023462_2014,1.0,1.0
1,00-0027944_2014,1.0,1.0
2,00-0026143_2014,1.0,1.0
3,00-0022736_2014,1.0,1.0
4,00-0026224_2014,2.0,2.0


In [69]:
coach_stats = coach_stats.merge(old_depth, how='left', left_on = 'gs_unq', right_on='gsis_unq')
coach_stats.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12825 entries, 0 to 12824
Data columns (total 139 columns):
 #    Column                         Non-Null Count  Dtype  
---   ------                         --------------  -----  
 0    gsis_id                        12825 non-null  object 
 1    cleaned_name_yahoo             12825 non-null  object 
 2    player_key                     12825 non-null  object 
 3    primary_position               12825 non-null  object 
 4    season                         12825 non-null  int64  
 5    gs_unq                         12825 non-null  object 
 6    bye_weeks.week                 12825 non-null  float64
 7    player_points.total            12825 non-null  float64
 8    recent_team                    12825 non-null  object 
 9    games                          12825 non-null  float64
 10   completions                    12825 non-null  float64
 11   attempts                       12825 non-null  float64
 12   passing_yards                 

In [70]:
coach_stats['gsis_unq'] = coach_stats['gsis_unq'].fillna(coach_stats['gs_unq'])
coach_stats['gsis_unq'].isna().sum()

np.int64(0)

In [83]:
coach_stats.loc[(coach_stats['wk1_depth'].isna())] = coach_stats.loc[(coach_stats['wk1_depth'].isna())].fillna(99)

In [84]:
coach_stats[coach_stats['cleaned_name_yahoo'] == 'steven jackson']

,gsis_id,cleaned_name_yahoo,player_key,primary_position,season,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,team,def_special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_safeties,def_misc_yards,def_fumble_recovery_opp,def_fumble_recovery_yards_opp,def_fumble_recovery_tds,def_punt_returns,def_punt_return_yards,def_kickoff_returns,def_kickoff_return_yards,HC,OC,DC,gsis_unq,wk1_depth,eos_depth
175,00-0022736,steven jackson,331.p.6783,RB,2014,00-0022736_2014,10.0,142.5,ATL,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190.0,707.0,6.0,0.0,0.0,38.0,-16.424412,0.0,20.0,27.0,148.0,0.0,0.0,0.0,33.0,121.0,6.0,1.621709,0.0,4.484848,0.043339,0.006740,0.069726,0.0,1.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,atlanta falcons,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mike smith,dirk koetter,mike nolan,00-0022736_2014,1.0,1.0
176,00-0022736,steven jackson,331.p.6783,RB,2014,00-0022736_2014,10.0,142.5,ATL,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190.0,707.0,6.0,0.0,0.0,38.0,-16.424412,0.0,20.0,27.0,148.0,0.0,0.0,0.0,33.0,121.0,6.0,1.621709,0.0,4.484848,0.043339,0.006740,0.069726,0.0,1.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,atlanta falcons,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mike smith,dirk koetter,mike nolan,00-0022736_2014,1.0,1.0
177,00-0022736,steven jackson,348.p.6783,RB,2015,00-0022736_2015,4.0,14.0,NE,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,50.0,1.0,0.0,0.0,4.0,-2.737325,0.0,1.0,1.0,20.0,0.0,0.0,0.0,-3.0,23.0,1.0,2.190686,0.0,-6.666667,0.001603,-0.000608,0.001978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,new england patriots,0

In [85]:
coach_stats.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12825 entries, 0 to 12824
Data columns (total 139 columns):
 #    Column                         Non-Null Count  Dtype  
---   ------                         --------------  -----  
 0    gsis_id                        12825 non-null  object 
 1    cleaned_name_yahoo             12825 non-null  object 
 2    player_key                     12825 non-null  object 
 3    primary_position               12825 non-null  object 
 4    season                         12825 non-null  int64  
 5    gs_unq                         12825 non-null  object 
 6    bye_weeks.week                 12825 non-null  float64
 7    player_points.total            12825 non-null  float64
 8    recent_team                    12825 non-null  object 
 9    games                          12825 non-null  float64
 10   completions                    12825 non-null  float64
 11   attempts                       12825 non-null  float64
 12   passing_yards                 

In [86]:
coach_stats.loc[(coach_stats['wk1_depth'].isna()) & (coach_stats['primary_position'] == 'DEF')] 

,gsis_id,cleaned_name_yahoo,player_key,primary_position,season,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,team,def_special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_safeties,def_misc_yards,def_fumble_recovery_opp,def_fumble_recovery_yards_opp,def_fumble_recovery_tds,def_punt_returns,def_punt_return_yards,def_kickoff_returns,def_kickoff_return_yards,HC,OC,DC,gsis_unq,wk1_depth,eos_depth


In [90]:
coach_stats[coach_stats.duplicated(keep=False)]

,gsis_id,cleaned_name_yahoo,player_key,primary_position,season,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,team,def_special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_safeties,def_misc_yards,def_fumble_recovery_opp,def_fumble_recovery_yards_opp,def_fumble_recovery_tds,def_punt_returns,def_punt_return_yards,def_kickoff_returns,def_kickoff_return_yards,HC,OC,DC,gsis_unq,wk1_depth,eos_depth
0,00-0004091,phil dawson,331.p.4269,K,2014,00-0004091_2014,4.0,125.00,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,25.0,31.0,5.0,1.0,55.0,0.806452,0.0,6.0,8.0,5.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948.0,250.0,45.0,33.0,33.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,san francisco 49ers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jim harbaugh,greg roman,vic fangio,00-0004091_2014,1.0,1.0
1,00-0004091,phil dawson,331.p.4269,K,2014,00-0004091_2014,4.0,125.00,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,25.0,31.0,5.0,1.0,55.0,0.806452,0.0,6.0,8.0,5.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948.0,250.0,45.0,33.0,33.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,san francisco 49ers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jim harbaugh,greg roman,vic fangio,00-0004091_2014,1.0,1.0
2,00-0004091,phil dawson,348.p.4269,K,2015,00-0004091_2015,9.0,102.00,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,24.0,27.0,1.0,2.0,54.0,0.888889,0.0,9.0,6.0,6.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30

In [92]:
coach_stats = coach_stats.drop_duplicates()
coach_stats.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 7773 entries, 0 to 12824
Data columns (total 139 columns):
 #    Column                         Non-Null Count  Dtype  
---   ------                         --------------  -----  
 0    gsis_id                        7773 non-null   object 
 1    cleaned_name_yahoo             7773 non-null   object 
 2    player_key                     7773 non-null   object 
 3    primary_position               7773 non-null   object 
 4    season                         7773 non-null   int64  
 5    gs_unq                         7773 non-null   object 
 6    bye_weeks.week                 7773 non-null   float64
 7    player_points.total            7773 non-null   float64
 8    recent_team                    7773 non-null   object 
 9    games                          7773 non-null   float64
 10   completions                    7773 non-null   float64
 11   attempts                       7773 non-null   float64
 12   passing_yards                  7773 

In [96]:
coach_stats['gs_unq'].nunique()

7720

In [97]:
coach_stats[coach_stats['gs_unq'].duplicated(keep=False)]

,gsis_id,cleaned_name_yahoo,player_key,primary_position,season,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,misc_yards,penalties,penalty_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,def_points_allowed,def_plays_against,def_yds_per_play,def_takeaways,def_cmp_against,def_pass_attempts_against,def_passing_allowed,def_passing_td_allowed,def_passing_ny_attempt,def_passing_1st,def_rushing_attamepts_against,def_rushing_allowed,def_rushing_td_allowed,def_rushing_ny_attempt,def_rushing_1st,def_penalties,def_penalty_yds,def_penalty_1st,def_scoring_drives%,def_TO_drives%,def_exp_pts,def_season,def_3Datt,def_3D%,def_4Datt,def_4D%,def_rz_att,def_rz%,team,def_special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_safeties,def_misc_yards,def_fumble_recovery_opp,def_fumble_recovery_yards_opp,def_fumble_recovery_tds,def_punt_returns,def_punt_return_yards,def_kickoff_returns,def_kickoff_return_yards,HC,OC,DC,gsis_unq,wk1_depth,eos_depth
28,00-0019536,shayne graham,331.p.5854,K,2014,00-0019536_2014,9.0,107.0,NO,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,19.0,22.0,3.0,0.0,50.0,0.863636,0.0,6.0,9.0,3.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,31;50;27;30;30;29;44;27;48;36;31;29;37;40;31;2...,41;42;51,0,636.0,134.0,0.0,46.0,47.0,0.0,1.0,0.978723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,new orleans saints,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sean payton,pete carmichael,rob ryan,00-0019536_2014,1.0,2.0
29,00-0019536,shayne graham,331.p.5854,K,2014,00-0019536_2014,9.0,107.0,NO,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,19.0,22.0,3.0,0.0,50.0,0.863636,0.0,6.0,9.0,3.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,31;50;27;30;30;29;44;27;48;36;31;29;37;40;31;2...,41;42;51,0,636.0,134.0,0.0,46.0,47.0,0.0,1.0,0.978723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,new orleans saints,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sean payton,pete carmichael,rob ryan,00-0019536_2014,2.0,2.0
302,00-0023367,nate washington,348.p.7492,WR,2015,00-0023367_2015,9.0,138.8,HOU,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.0,94.0,658.0,4.0,0.0,0.0,1067.0,162.0,33.0,13.629010,0.0,0.616682,0.153094,0.179660,0.355404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.

In [99]:
# Separate wk1_depth from other columns
other_cols = [col for col in coach_stats.columns if col != 'wk1_depth']

# Group and aggregate
coach_stats = (coach_stats.groupby('gsis_unq', as_index=False)
               .agg({'wk1_depth': 'mean', 
                     **{col: 'first' for col in other_cols if col != 'gsis_unq'}}))

In [102]:
coach_stats.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7720 entries, 0 to 7719
Data columns (total 139 columns):
 #    Column                         Non-Null Count  Dtype  
---   ------                         --------------  -----  
 0    gsis_unq                       7720 non-null   object 
 1    wk1_depth                      7720 non-null   float64
 2    gsis_id                        7720 non-null   object 
 3    cleaned_name_yahoo             7720 non-null   object 
 4    player_key                     7720 non-null   object 
 5    primary_position               7720 non-null   object 
 6    season                         7720 non-null   int64  
 7    gs_unq                         7720 non-null   object 
 8    bye_weeks.week                 7720 non-null   float64
 9    player_points.total            7720 non-null   float64
 10   recent_team                    7720 non-null   object 
 11   games                          7720 non-null   float64
 12   completions                    7

In [104]:
coach_stats[coach_stats['season'] <= 2024].to_csv('data14to24.csv', index=False)